In [1]:
import numpy as np
import os
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
from skimage import color, transform
from pathlib import Path
import keras
from PIL import Image
import tensorflow as tf
from sklearn.base import BaseEstimator, TransformerMixin

2025-10-25 15:52:53.712534: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-25 15:52:53.957105: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-25 15:52:56.006178: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


## Loading Saved Model

In [2]:
curr_dir = os.getcwd()
curr_dir

'/home/root123/GitHub/Two_Months_ML_Journey/Week 4'

In [3]:
cnn = keras.saving.load_model(f'{curr_dir}/saved_models/cnn_scratch_model.keras')

I0000 00:00:1761386882.973982    3689 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1767 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


## Creating Pipeline

In [4]:
# # creating a function transformer to pass it to pipeline

# def preprocess(img):
#     # Convert PIL Image → NumPy array
#     if isinstance(img, Image.Image):
#         img = np.array(img)
#     # grayscale (no channel dimension)
#     if img.ndim == 2:
#         img = color.gray2rgb(img)
#         print('Gray to RGB')
#     # RGBA (4 channels)
#     elif img.shape[2] == 4:
#         img = color.rgba2rgb(img)
#         print('RGBA to RGB')

#     # Resize to 128x128
#     resized_img = transform.resize(img, (128, 128), anti_aliasing=True) # anti_aliasing makes img smoother when shrinked
#     resized_img = resized_img.astype('float32')
#     norm_resized_img = resized_img / 255.0
#     return norm_resized_img

# # converting our function to 'Function Transformer' for compatibility with Sklearn's pipeline
# rgb_resize_transformer = FunctionTransformer(
#     lambda imgs: np.array([preprocess(img) for img in imgs])
# )

In [5]:
class KerasPreprocessorWrapper(BaseEstimator, TransformerMixin):
    def __init__(self, target_size=(128, 128)):
        self.target_size = target_size

    def fit(self, X, y=None):
        # no learning from data here, but required by sklearn's API
        return self

    def transform(self, X):
        
            # Convert PIL images to numpy arrays if needed
        if isinstance(X, list):
            X = [np.array(img) if hasattr(img, 'size') else img for img in X]
            X = np.stack(X, axis=0)  # shape: (batch, H, W, C)
        elif isinstance(X, Image.Image):  # single image
            X = np.expand_dims(np.array(X), axis=0)
        # Ensure the input is a TensorFlow tensor
        X = tf.convert_to_tensor(X)

        # Normalize dtype and scale to [0,1]
        X = tf.image.convert_image_dtype(X, tf.float32)

        # Handle grayscale (1 channel) or RGBA (4 channels)
        num_channels = tf.shape(X)[-1]

        def to_rgb_if_grayscale():
            return tf.image.grayscale_to_rgb(X)

        def to_rgb_if_rgba():
            return X[..., :3]

        def identity():
            return X

        X = tf.case(
            [
                (tf.equal(num_channels, 1), to_rgb_if_grayscale),
                (tf.equal(num_channels, 4), to_rgb_if_rgba)
            ],
            default=identity,
            exclusive=True
        )

        # Resize to target size
        X = tf.image.resize(X, self.target_size)

        # Return a TensorFlow tensor (not NumPy)
        return X

In [6]:
pipeline = Pipeline(steps=[
    ('preprocess', KerasPreprocessorWrapper()),
    ('model', cnn),
])

In [7]:
destination_path = 'test_data'
predict_img_path = curr_dir/Path(destination_path)
predictions = [
    'Pepper Bell Bacterial Spot',
    'Pepper Bell Healthy',
    'Potato Early Blight',
    'Potato Healthy',
    'Potato Late Blight',
    'Tomato Target Spot',
    'Tomato Mosaic Virus',
    'Tomato Yellow Leaf Curl Virus',
    'Tomato Bacterial Spot',
    'Tomato Early Blight',
    'Tomato Healthy',
    'Tomato Late Blight',
    'Tomato Leaf Mold',
    'Tomato Septoria Leaf Spot',
    'Tomato Spider Mites',
]   


In [9]:
for img_file in predict_img_path.iterdir():
    if img_file.suffix.lower() in ['.jpg', '.png']:
        # Open image
        img = Image.open(img_file)
        
        # Run through your pipeline (wrap in list or array)
        pred = pipeline.predict([img])
        print(pred[0])
        print(f'Prediction: {predictions[np.argmax(pred)]}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
[9.9940312e-01 5.9683027e-04 6.4497718e-10 5.3543253e-10 1.1160387e-14
 7.0877753e-16 7.6820848e-11 3.9348008e-10 4.0131235e-23 2.7700147e-14
 8.8239543e-28 1.2089531e-18 4.5733163e-23 0.0000000e+00 2.6475101e-22]
Prediction: Pepper Bell Bacterial Spot
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
[2.7851493e-04 4.4583013e-16 9.4973463e-01 7.5882529e-03 3.1538099e-15
 5.7341404e-10 5.6694695e-03 2.9872486e-02 1.7974696e-08 6.8566832e-03
 2.2041774e-14 4.1508645e-12 2.1156896e-14 4.7882338e-20 6.6638628e-09]
Prediction: Potato Early Blight
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
[4.9699157e-14 4.0359301e-14 1.8305577e-16 7.5516850e-13 2.2512219e-15
 1.4591925e-11 2.8298296e-06 7.2362326e-07 3.6750422e-11 5.0048136e-08
 1.9972555e-05 9.9997640e-01 2.1149343e-13 6.8599968e-17 9.6379274e-08]
Prediction: Tomato Late Blight
